In [1]:
import pandas as pd
from io import StringIO
from pygeofilter_aeronet.evaluator import (
    to_aeronet_api,
    to_aeronet_api_querystring,
    http_invoke,
)

## Use cql2-json filters to interact with the AERONET API 

See https://aeronet.gsfc.nasa.gov/print_web_data_help_v3.html for details

Decisions:

- AVG - Data Format, All points: AVG=10, use `[{"property": "data_format"}, "all-points"]`
- AVG - Data Format, Daily average: AVG=20, use `[{"property": "data_format"}, "daily-average"]`



In [2]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2023-02-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2023-02-28T23:59:59Z"},
            ],
        },
    ],
}

Print the AERONET API querystring:

In [3]:
to_aeronet_api(cql2_filter=cql2_filter)

'site=Cart_Site&AOD10=1&if_no_html=1&AVG=20&year=2023&month=2&day=1&hour=0&minute=0&year2=2023&month2=2&day2=28&hour2=23&minute2=59'

Dry-run the HTTP request:

In [4]:
http_invoke(cql2_filter=cql2_filter, dry_run=True)

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&AOD10=1&if_no_html=1&AVG=20&year=2023&month=2&day=1&hour=0&minute=0&year2=2023&month2=2&day2=28&hour2=23&minute2=59


{}

## Examples

### Level 1.0 data from the "Cart_Site" for AOD daily averages

Define a cql2-json filter for:

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD10=1&AVG=20



In [5]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

In [6]:
raw_data = http_invoke(cql2_filter=cql2_filter, dry_run=False)

In [7]:
df = pd.read_csv(StringIO(raw_data), skiprows=5)

In [8]:
df.head(5)

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[440-675_Angstrom_Exponent],N[500-870_Angstrom_Exponent],N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m)
0,Cart_Site,31:05:2000,12:00:00,152,-999.0,0.035949,0.052400,-999.0,-999.0,0.063064,...,35,35,35,0,lev10,99,Cart_Site,36.60667,-97.48639,318.0
1,Cart_Site,01:06:2000,12:00:00,153,-999.0,0.177293,0.190211,-999.0,-999.0,0.198619,...,31,31,31,0,lev10,99,Cart_Site,36.60667,-97.48639,318.0
2,Cart_Site,03:06:2000,12:00:00,155,-999.0,0.098948,0.113883,-999.0,-999.0,0.134527,...,6,6,6,0,lev10,99,Cart_Site,36.60667,-97.48639,318.0
3,Cart_Site,04:06:2000,12:00:00,156,-999.0,0.113260,0.138564,-999.0,-999.0,0.176712,...,7,7,7,0,lev10,99,Cart_Site,36.60667,-97.48639,318.0
4,Cart_Site,05:06:2000,12:00:00,157,-999.0,0.031582,0.042280,-999.0,-999.0,0.046784,...,55,55,55,0,lev10,99,Cart_Site,36.60667,-97.48639,318.0


## Level 1.5 data from all sites with available data for all AOD points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD15=1&AVG=10


In [9]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "AOD15"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "all-points"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

In [10]:
http_invoke(cql2_filter=cql2_filter, dry_run=True)

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?AOD15=1&if_no_html=1&AVG=10&year=2000&month=6&day=1&hour=0&minute=0&year2=2000&month2=6&day2=14&hour2=23&minute2=59


{}


## Level 2.0 data from the "Cart_Site" for all AOD points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD20=1&AVG=10


In [11]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

In [12]:
http_invoke(cql2_filter=cql2_filter, dry_run=True)

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&AOD20=1&if_no_html=1&AVG=20&year=2000&month=6&day=1&hour=0&minute=0&year2=2000&month2=6&day2=14&hour2=23&minute2=59


{}


## Level 2.0 data from the "Cart_Site" for all SDA points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&SDA20=1&AVG=10



In [13]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "SDA20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

In [14]:
http_invoke(cql2_filter=cql2_filter, dry_run=True)

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&SDA20=1&if_no_html=1&AVG=20&year=2000&month=6&day=1&hour=0&minute=0&year2=2000&month2=6&day2=14&hour2=23&minute2=59


{}

## Geographical search


In [23]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "SDA20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [8.0, 44.0],
                            [14.0, 44.0],
                            [14.0, 48.0],
                            [8.0, 48.0],
                            [8.0, 44.0],
                        ]
                    ],
                },
            ],
        },
    ],
}

In [25]:
raw_data = http_invoke(cql2_filter=cql2_filter, dry_run=False)

In [26]:
df = pd.read_csv(StringIO(raw_data), skiprows=5)

In [27]:
df

,AERONET_Site,Date_(dd:mm:yyyy),Time_(hh:mm:ss),Day_of_Year,Total_AOD_500nm[tau_a],Fine_Mode_AOD_500nm[tau_f],Coarse_Mode_AOD_500nm[tau_c],FineModeFraction_500nm[eta],2nd_Order_Reg_Fit_Error-Total_AOD_500nm[regression_dtau_a],RMSE_Fine_Mode_AOD_500nm[Dtau_f],...,N[dAE/dln(wavelength)-Total_500nm[alphap]],N[AE-Fine_Mode_500nm[alpha_f]],N[dAE/dln(wavelength)-Fine_Mode_500nm[alphap_f]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),Unnamed: 34
0,Venise,31:05:2000,12:00:00,152,0.702868,0.691315,0.011553,0.983048,0.004935,0.201480,...,23,23,23,lev20,112,Venise,45.31390,12.5083,10.0,NaN
1,Venise,01:06:2000,12:00:00,153,0.240019,0.231789,0.008230,0.950866,0.003788,0.043439,...,43,43,43,lev20,112,Venise,45.31390,12.5083,10.0,NaN
2,Venise,02:06:2000,12:00:00,154,0.409857,0.398926,0.010930,0.972578,0.004475,0.078353,...,46,46,46,lev20,112,Venise,45.31390,12.5083,10.0,NaN
3,Venise,03:06:2000,12:00:00,155,0.569970,0.556496,0.013475,0.976083,0.004998,0.106529,...,50,50,50,lev20,112,Venise,45.31390,12.5083,10.0,NaN
4,Venise,04:06:2000,12:00:00,156,0.505405,0.485035,0.020370,0.953210,0.005549,0.090719,...,40,40,40,lev20,112,Venise,45.31390,12.5083,10.0,NaN
5,Venise,05:06:2000,12:00:00,157,0.475985,0.454433,0.021552,0.937115,0.005355,0.092959,...,60,60,60,lev20,112,Venise,45.31390,12.5083,10.0,NaN
6,Venise,06:06:2000,12:00:00,158,0.509634,0.372044,0.137591,0.749999,0.006624,0.066462,...,18,18,18,lev20,112,Venise,45.31390,12.5083,10.0,NaN
7,Venise,07:06:2000,12:00:00,159,0.245020,0.207304,0.037716,0.842297,0.004223,0.040453,...,31,31,31,lev20,112,Venise,45.31390,12.5083,10.0,NaN
8,Venise,08:06:2000,12:00:00,160,0.194982,0.175087,0.019896,0.894646,0.003871,0.033336,...,60,60,60,lev20,112,Venise,45.31390,12.5083,10.0,NaN
9,Venise,09:06:2000,12:00:00,161,0.289063,0.250073,0.038990,0.860046,0.004191,0.045663,...,62,62,62,lev20,112,Venise,45.31390,12.5083,10.0,NaN
